In [ ]:
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth
import spotipy
import sys
from pprint import pprint
from pymonad.tools import curry
from pymonad.reader import Compose
import pymonad.tools
from oslash import Just, Nothing, List
import functools
import operator

In [ ]:
client_id = "01f2938c5262491da9b4ccfc11aea077"
client_secret = "978c8f2e99f84ad99a7f9f13ac59a3fa"
sp = spotipy.Spotify(client_credentials_manager = SpotifyClientCredentials(client_id, client_secret))

#### 1.- Obtener 20 canciones de un artista dado por el usuario.

In [ ]:
def generador(author:str):
    result = sp.search(author, limit=20)
    for i in range(len(result["tracks"]["items"])):
        yield result["tracks"]["items"][i]["name"]

if __name__ == '__main__':
    author = input("Ingrese el nombre del autor: ")
    for i,j in enumerate(generador(author)):
        print(i+1,": ",j)

#### 2.- Obtener la cancion mas larga de las primeras 20 canciones de un artista dado por el usuario.

In [ ]:
def oneover(x):
    try:
        ret = x[1]/60000
        print("La cancion mas larga encontrada es: ",x[0])
    except:
        return Nothing()
    return Just(ret)

def leer(autor:str):
    result = sp.search(autor, limit=20)
    listaAux = list(map(lambda x :[x["name"], x["duration_ms"]], result["tracks"]["items"]))
    def maxL():
        aux2 = max(listaAux, key=lambda x : x[1])
        a = Just(aux2).bind(oneover)
        print("Con una duracion de: ",a)
    return maxL

if __name__ == '__main__':
    author = input("Ingrese el nombre del autor: ")
    r = leer(author)
    r()

#### 3.- Crea una tupla de tuplas con los generos: banda, rock, electronica y 20 canciones de cada genero.

In [ ]:
def generador(genre:str)->list:
    result = sp.search(q = 'genre' + genre, limit = 20)
    return list(map(lambda x : x["name"], result["tracks"]["items"]))

if __name__ == '__main__':
    generos= ["Banda", "Rock", "Electronica"]
    canciones = list(map(lambda x : generador(x), generos))
    final= tuple(zip(generos, canciones))
    print(final)

#### 4.- Obten el tiempo total de las primeras 30 canciones de un artista dado por el usuario.

In [ ]:
def fun(author:str)->list:
    result = sp.search(author, limit=30)
    aux2= []
    return list(map(lambda x: x["duration_ms"], result["tracks"]["items"]))

if __name__ == '__main__':
    author = input("Ingrese el nombre del autor: ")
    r = fun(author)
    r2= functools.reduce(operator.add,r)/ 60000
    final = Just(r2)
    print(final)

#### 5.- Obtener 10 canciones de un artista dado por el usuario.

In [ ]:
def read_artist()->str:
    l = input('Ingrese un artista: ')
    return l

def search1(la:str)-> list:
    b = sp.search(la, limit=10)
    return list(map(lambda x: x["name"], b["tracks"]["items"]))

print(search1(read_artist()))

#### 6.- Crear un conjunto donde las canciones del top 10 de luis miguel no esten en las canciones generales de ese artista

In [ ]:
def dif(a:str, u:str):
    b = sp.search(a, limit=10)
    lista = list(map(lambda x: [0, x["name"]], b["tracks"]["items"]))
    
    response = sp.artist_top_tracks(u)
    lista += list(map(lambda x: [1, x["name"]], response["tracks"]))
    return lista

urn = 'spotify:artist:2nszmSgqreHSdJA3zWPyrW'
c1 = set()
c2 = set()
lista2 = dif('Luis Miguel', urn)
c1 = {x[1] for x in filter(lambda x: x[0] == 0, lista2)}
c2 = {x[1] for x in filter(lambda x: x[0] != 0, lista2)}
# print(c1)
# print(c2)
c3 = c2.symmetric_difference(c2.intersection(c1))
print(c3)

#### 7.- Mostrar una lista de canciones del albúm "00:00" del artista "Siddhartha", pero eliminando la primera canción

In [ ]:
def tail(x:list)->list:
    if x:
        del x[0]

urn = 'spotify:album:371qTqzTdc0reynwwfM2Bt'
a = sp.album(urn)
a1 = list(map(lambda x: x["name"], a["tracks"]["items"]))
# print(a1)
tail(a1)
print(a1)

#### 8.- Mostrar una lista con los géneros del artista "José José"

In [ ]:
urn = 'spotify:artist:4mN0qcMxWX8oToqfDPM5yV'
def closer(a):
    def genres(g):
        artist = list(map(lambda x : x, sp.artist(a)[g]))
        print(artist)
    return genres

c = closer(urn)
c('genres')

#### 9.- Promedio de duración en ms de las primeras 30 canciones de un artista dado por el usuario.

In [ ]:
@curry(1)
def sumc(x):
    try:
        return sum(x)
    except:
        return Nothing()

def oneover(x):
    try:
        ret = x[0]/x[1]
    except:
        return Nothing()
    return Just(ret)

def fun(author:str)->list:
    result = sp.search(author, limit=30)
    aux2 = list(map(lambda x: x["duration_ms"], result["tracks"]["items"]))
    return sumc(aux2), len(result["tracks"]["items"])

g = input('Ingrese un artista: ')
a =Just(fun(g)).bind(oneover)
print(a)

#### 10.- Obtener los artistas de 50 canciones en general que colaboraron en un artista dado por el usuario.

In [ ]:
def generador(author:str):
    result = sp.search(author, limit=50)
    return list(map(lambda x: x["artists"], result["tracks"]["items"]))
    
def artistas(it):
    aux = []
    for i in it:
        aux += list(map(lambda x : x["name"], i))
    return(set(aux))

if __name__ == '__main__':
    autor = input("Ingrese el nombre del autor: ")
    print(artistas(generador(autor)))

#### 11.- Obtener un diccionario de diccionarios donde se muestre a los artistas con una lista de 30 de sus canciones

In [ ]:
autores = []
dicc = {}
n = int(input('Ingrese el número de artistas: '))
for i in range (n):
    autores.append(input('Ingrese el nombre del artista: '))
tupla = [{autor:list(map(lambda x : x["name"], sp.search(autor, limit=30)["tracks"]["items"]))} for autor in autores]
for x, t in enumerate(tupla, start = 1):
    dicc['Cantante '+ str(x)] = t
print(dicc)
